# ABS Quarterly Financial Accounts 5232

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Household-balance-sheet" data-toc-modified-id="Household-balance-sheet-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Household balance sheet</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys
import platform
import psutil
from pathlib import Path
import calendar
import re

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# abs specific imports
from abs_common import (get_ABS_meta_and_data,
                        get_identifier,
                        finalise_plot,
                        plot_Qgrowth)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
# cache directory
CACHE_DIR = '../cache'
Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

# charts directory
CHART_DIR = '../charts/5232'
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)

## Python environment

In [3]:
N = 50
print('-'*N)
print(f"System:         {platform.system()}")
print(f"Release:        {platform.release()}")
#print(f"Version:        {platform.version()}")
print(f"Machine:        {platform.machine()}")
print(f"Processor:      {platform.processor()}")
print(f"RAM:            {round(psutil.virtual_memory().total / 1024**3)}GB")
print('-'*N)
print(f"Python:         {platform.python_version()}")
print(f"Psutil:         {psutil.__version__}")
print(f"Pandas:         {pd.__version__}")
print(f"Numpy:          {np.__version__}")
print(f"Matplotlib:     {mpl.__version__}")
print('-'*N)

--------------------------------------------------
System:         Darwin
Release:        20.1.0
Machine:        x86_64
Processor:      i386
RAM:            64GB
--------------------------------------------------
Python:         3.8.3
Psutil:         5.7.0
Pandas:         1.0.5
Numpy:          1.18.5
Matplotlib:     3.2.2
--------------------------------------------------


## Get data from ABS

In [4]:
url_template = ('https://www.abs.gov.au/statistics/economy/national-accounts/'
                'australian-national-accounts-finance-and-wealth/MONTH-YEAR/'
                'All_Time_Series_Workbook.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

We need to cache this file
File for jun-2020 of size 5.9 MB


## Plot

### Set-up some constants we will use across all plots

In [5]:
RECENT = data.index.max() - pd.DateOffset(years=3)
plot_times = (None, RECENT)
plot_frames = (data.copy(), data[data.index >= RECENT].copy())
plot_tags = ('full', 'recent')

CVM = 'Chain volume measures'
source = 'Source: ABS 5232 table'

### Household balance sheet

In [13]:
table = '34'
series_type = 'Original'
household_assets = ('Amounts outstanding at end of period ;  '
                    'Assets ;  Total financial assets ;  '
                    'Total (Counterparty sectors) ;')
household_liabilities = ('Amounts outstanding at end of period ;  '
                         'Liabilities ;  Total financial liabilities ;  '
                         'Total (Counterparty sectors) ;')

assets_id, units = get_identifier(meta, household_assets,
                               series_type, table)
liabilities_id, _ = get_identifier(meta, household_liabilities,
                               series_type, table)

for plot_frame, plot_tag in zip(plot_frames, plot_tags):
    
    # get data
    assets = plot_frame[assets_id]
    liabilities = plot_frame[liabilities_id]
    net = assets - liabilities

    # plot
    dataset = assets, liabilities, net
    desc = 'Assets', 'Liabilities', 'Balance Sheet'
    for series, name in zip(dataset, desc):
        ax = series.plot(lw=2)
        title = f'Household: {name}'
        finalise_plot(ax, title, units, 
                      plot_tag, CHART_DIR,
                      rfooter=f'{source} {table}',
                      lfooter=f'{series_type.capitalize()} series')

## Finished

In [14]:
print('Finished')

Finished
